### statically-typed language : java,C, C++
 컴파일시에 에러가 검출된다. 타입구분이 명확한것들을 정적타입 랭귀지 라함.
 
### dynamic-typed language : python, javascript, Ruby... 
 인터프리터 언어라고도한다. 타입에 딱히 신경쓰지않아도 개발이가능하며 개발속도 기본적으로 빠르다.
 
IDE에서 statical language 는 코드 suggestion이 잘 적용되지만, python같은 dynamic language 는 지원이 상대적으로 부족하다.

특히나 modern IDE에서는 관용구나 표현식을 제공하지않는다.


In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob

In [2]:
data_path='data_samples/mapping.map'
data_pattern = 'all_{-type-}_data.dat'
hidden_size = 10
seq_length = 10
batch_size = 10
num_samples = 10 # 원래는 100개
with open(data_path, "rb") as f:
        word_to_id = pickle.load(f)
vocab_size = len(word_to_id)

In [3]:
data_pattern = data_pattern.replace("{-type-}", "train") + ".part*"
data_pattern

'all_train_data.dat.part*'

In [4]:
train_files = [
    'D:\Atlassian\git\tensorflowexam\paper_implements\data_samples\affinelayer\pix2pix-tensorflow\tools\download-dataset.py',
'D:\Atlassian\git\tensorflowexam\paper_implements\data_samples\affinelayer\pix2pix-tensorflow\tools\process.py']

In [5]:
cell = tf.nn.rnn_cell.BasicLSTMCell(
    hidden_size, forget_bias=1.0, state_is_tuple=True)

In [6]:
cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.8)
actual_lengths = tf.placeholder(tf.int32, [batch_size], name="actual_lengths")
initial_state = cell.zero_state(batch_size, tf.float32)
targets = tf.placeholder(tf.int32, [seq_length, batch_size], name="targets")

In [7]:
input_data = tf.placeholder(tf.int32, [seq_length, batch_size], name="inputs")

with tf.device('/cpu:0'):
    _embedding = embedding = tf.get_variable("embedding", [vocab_size, hidden_size],
                                                  trainable=True)

    inputs = tf.gather(embedding, input_data)

In [8]:
embedding, input_data, inputs

(<tf.Variable 'embedding:0' shape=(1458, 10) dtype=float32_ref>,
 <tf.Tensor 'inputs:0' shape=(10, 10) dtype=int32>,
 <tf.Tensor 'Gather:0' shape=(10, 10, 10) dtype=float32>)

In [9]:
def output_and_loss(cell, inputs):
    output, state = tf.nn.dynamic_rnn(
        cell, inputs, sequence_length=actual_lengths,
        initial_state=initial_state)

    output = tf.reshape(output, [-1, hidden_size])
    softmax_w = tf.get_variable("softmax_w", [hidden_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    logits = tf.matmul(output, softmax_w) + softmax_b
    predict = tf.nn.softmax(logits)

    labels = tf.reshape(targets, [batch_size * seq_length, 1])

    loss = tf.nn.sampled_softmax_loss(
        tf.transpose(softmax_w), softmax_b, output, labels, num_samples, vocab_size) \
        if num_samples > 0 else \
        tf.nn.sparse_softmax_cross_entropy_with_logits(_logits, tf.reshape(targets, [-1]))

    return logits, predict, loss, state
_logits, _predict, _loss, _final_state=output_and_loss(cell,inputs)

TypeError: Input 'y' of 'Mul' Op has type float32 that does not match type int32 of argument 'x'.